In [9]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.rcParams['lines.linewidth'] = 2

data = pd.read_csv("Cu(ny).txt")


x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

slope, intercept, r, p, std_err = stats.linregress(x, y)

print(slope)
print(intercept)

2.0
0.0


In [10]:
data

,number of plates,thickness (mm),Net area,error,live time
0,0.0,0,0,0.00,NaN
1,4.5,90264,750,52.56,NaN
2,4.8,73246,646,54.00,NaN
3,4.8,55818,569,54.96,NaN
4,4.8,1161,688,55.70,NaN
5,4.8,-561,668,56.28,NaN


In [ ]:
plt.figure(figsize=[15, 7])
plt.plot(x, y, label="log of data")
plt.plot(x, x*slope, label="fit")